# Setup

Imports, helper functions and globals.

In [90]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from scipy.interpolate import interp1d

In [92]:
# Globals.
SAMPLE_RATE = 250
DATA_PATH = "../data"

# Helpers.
def clean_data(df):
    # Extract participant ID no.
    pid = df['PID'][0]
    
    # Remove extrananeous rows/columns.
    df = df.drop([0])
    df = df.drop(columns=['tags', 'PID'])
    return df, pid

def resample_cursor_data(time_series, sample_rate=SAMPLE_RATE):
    time_ms = [event['timeMs'] for event in time_series]
    valence = [event['valence'] for event in time_series]
    arousal = [event['arousal'] for event in time_series]
    
    time_ms.insert(0, 0.0)
    valence.insert(0, 0.0)
    arousal.insert(0, 0.0)
    
    valence_func = interp1d(time_ms, valence)
    arousal_func = interp1d(time_ms, arousal)
    
    period = 1/SAMPLE_RATE
    resample_time = np.arange(0, time_ms[-1], period)
    
    valence = valence_func(resample_time)
    arousal = arousal_func(resample_time)
    
    time_s = resample_time / 1000
    
    return time_s, valence, arousal

def my_plot(time, valence, arousal, show=True):
    plt.plot(time, valence)
    plt.plot(time, arousal)
    plt.xlabel('Time (s)')
    plt.ylabel('Normalized value')
    plt.legend({'Valence', 'Arousal'})
    if show:
        plt.show()

# Analysis

Sample analysis below. Specify filename and trial number to visualize valence and arousal values over time.

In [94]:
# Read test data.
filename = "PID_14__2021-9-5_10h17-36.json"
trial_no = 12;

# Get filepath.
filepath = os.path.join(DATA_PATH, filename)

# Load and clean data.
df = pd.read_json(filepath)
df, pid = clean_data(df)

# Extract trial and resample cursor data.
tmp = df.iloc[trial_no - 1]['time_series']
time, valence, arousal = resample_cursor_data(tmp)

my_plot(time, valence, arousal)